In [256]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from sklearn.metrics import classification_report
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [257]:
df_elasticidad = pd.read_pickle("../output/base_etapa_3.pkl")

In [258]:
# Creamos una columna para simular el costo
df_elasticidad['costo'] = 17
df_elasticidad['costo_seg0'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg0']))
df_elasticidad['costo_seg1'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg1']))
df_elasticidad['costo_seg2'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg2']))
# df_elasticidad['costo_seg3'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg3']))
# df_elasticidad['costo_seg4'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg4']))

In [259]:
df_elasticidad['ingresos_seg0'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg0']))
df_elasticidad['ingresos_seg1'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg1']))
df_elasticidad['ingresos_seg2'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg2']))
# df_elasticidad['ingresos_seg3'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg3']))
# df_elasticidad['ingresos_seg4'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg4']))

In [306]:
# Calculamos la rentabilidad
df_elasticidad['rentabilidad_seg0'] = df_elasticidad['ingresos_seg0']- df_elasticidad["costo_seg0"]
df_elasticidad['rentabilidad_seg1'] = df_elasticidad['ingresos_seg1'] - df_elasticidad["costo_seg1"]
df_elasticidad['rentabilidad_seg2'] = df_elasticidad['ingresos_seg2'] - df_elasticidad["costo_seg2"]
# df_elasticidad['rentabilidad_seg3'] = df_elasticidad['ingresos_seg3'] - df_elasticidad["costo_seg3"]
# df_elasticidad['rentabilidad_seg4'] = df_elasticidad['ingresos_seg4'] - df_elasticidad["costo_seg4"]

# df_elasticidad['rentabilidad_seg0'] = df_elasticidad['ingresos_seg0'] - df_elasticidad["costo"]
# df_elasticidad['rentabilidad_seg1'] = df_elasticidad['ingresos_seg1'] - df_elasticidad["costo"]
# df_elasticidad['rentabilidad_seg2'] = df_elasticidad['ingresos_seg2'] - df_elasticidad["costo"]
# df_elasticidad['rentabilidad_seg3'] = df_elasticidad['ingresos_seg3'] - df_elasticidad["costo"]

In [307]:
df_elasticidad

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,costo,costo_seg0,costo_seg1,costo_seg2,ingresos_seg0,ingresos_seg1,ingresos_seg2,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2
0,16.00,2.452854,2.287199,2.630060,17,12.706415,10.422813,15.482908,12.033705,9.912935,14.601325,-0.672711,-0.509878,-0.881583
1,16.58,2.269261,2.125851,2.417276,17,10.192873,8.471696,12.191156,9.986050,8.313363,11.924921,-0.206823,-0.158333,-0.266235
2,17.16,2.105034,1.980862,2.228161,17,8.238762,6.935075,9.678475,8.296409,6.979624,9.751387,0.057647,0.044549,0.072912
3,17.74,1.957575,1.850101,2.059412,17,6.706566,5.715069,7.742858,6.901284,5.866868,7.984899,0.194718,0.151799,0.242042
4,18.32,1.824704,1.731772,1.908272,17,5.495581,4.739095,6.238979,5.748381,4.937758,6.547866,0.252800,0.198663,0.308887
5,18.89,1.706554,1.626123,1.774654,17,4.546170,3.964991,5.079162,4.809673,4.173514,5.395770,0.263504,0.208523,0.316608
6,19.47,1.597440,1.528164,1.651946,17,3.770035,3.325011,4.145993,4.017374,3.521907,4.438152,0.247339,0.196896,0.292159
7,20.05,1.498206,1.438729,1.540960,17,3.143634,2.802775,3.404499,3.360501,2.976167,3.656299,0.216867,0.173392,0.251800
8,20.63,1.407709,1.356860,1.440285,17,2.634929,2.374104,2.811372,2.815212,2.518544,3.017056,0.180283,0.144440,0.205684
9,21.21,1.324966,1.281729,1.348712,17,2.219380,2.020273,2.333935,2.361950,2.134327,2.493644,0.142569,0.114054,0.159710


In [294]:
# Graficamos las curvas de Precio-Ingresos
fig = px.line(df_elasticidad, x='irr', y=['ingresos_seg0', 'ingresos_seg1',
                                             'ingresos_seg2',# 'ingresos_seg3', 'ingresos_seg4'
                                             ],
              labels={'irr':'tasa', 'value':'Ingresos'},
              title='Curva de Ingresos',
              width=800, height=600)

fig.show()

In [295]:
# Graficamos las curvas de Precio-Rentabilidad
fig = px.line(df_elasticidad, x='irr', y=['rentabilidad_seg0',
                                             'rentabilidad_seg1',
                                             'rentabilidad_seg2',
                                             #'rentabilidad_seg3',
                                             #'rentabilidad_seg4',
                                             ],
              labels={'irr':'tasa', 'value':'Rentabilidad'},
              title='Curvas de Rentabilidad',
              width=800, height=600)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="red", opacity=0.5)

fig.show()

In [296]:
df_elasticidad.head()

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,costo,costo_seg0,costo_seg1,costo_seg2,ingresos_seg0,ingresos_seg1,ingresos_seg2,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2
0,16.00,2.452854,2.287199,2.630060,17,12.706415,10.422813,15.482908,12.033705,9.912935,14.601325,-0.746333,-0.822562,-1.500899
1,16.58,2.269261,2.125851,2.417276,17,10.192873,8.471696,12.191156,9.986050,8.313363,11.924921,-0.260890,-0.412484,-0.753881
2,17.16,2.105034,1.980862,2.228161,17,8.238762,6.935075,9.678475,8.296409,6.979624,9.751387,0.017318,-0.163503,-0.314227
3,17.74,1.957575,1.850101,2.059412,17,6.706566,5.715069,7.742858,6.901284,5.866868,7.984899,0.164106,-0.019653,-0.067673
4,18.32,1.824704,1.731772,1.908272,17,5.495581,4.739095,6.238979,5.748381,4.937758,6.547866,0.229115,0.056490,0.059328


In [297]:
# Hallamos los precios que maximicen los ingresos en cada segmento
max_ing_seg0 = df_elasticidad.loc[(df_elasticidad['ingresos_seg0'] == df_elasticidad['ingresos_seg0'].max())]
max_ing_seg1 = df_elasticidad.loc[(df_elasticidad['ingresos_seg1'] == df_elasticidad['ingresos_seg1'].max())]
max_ing_seg2 = df_elasticidad.loc[(df_elasticidad['ingresos_seg2'] == df_elasticidad['ingresos_seg2'].max())]
# max_ing_seg3 = df_elasticidad.loc[(df_elasticidad['ingresos_seg3'] == df_elasticidad['ingresos_seg3'].max())]
# max_ing_seg4 = df_elasticidad.loc[(df_elasticidad['ingresos_seg4'] == df_elasticidad['ingresos_seg4'].max())]

In [298]:
max_ing_seg0

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,costo,costo_seg0,costo_seg1,costo_seg2,ingresos_seg0,ingresos_seg1,ingresos_seg2,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2
0,16.0,2.452854,2.287199,2.63006,17,12.706415,10.422813,15.482908,12.033705,9.912935,14.601325,-0.746333,-0.822562,-1.500899


In [299]:
max_ing_seg1

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,costo,costo_seg0,costo_seg1,costo_seg2,ingresos_seg0,ingresos_seg1,ingresos_seg2,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2
0,16.0,2.452854,2.287199,2.63006,17,12.706415,10.422813,15.482908,12.033705,9.912935,14.601325,-0.746333,-0.822562,-1.500899


In [300]:
# Hallamos los precios que maximicen la rentabilidad en cada segmento
max_ren_seg0 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg0'] == df_elasticidad['rentabilidad_seg0'].max())]
max_ren_seg1 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg1'] == df_elasticidad['rentabilidad_seg1'].max())]
max_ren_seg2 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg2'] == df_elasticidad['rentabilidad_seg2'].max())]
# max_ren_seg3 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg3'] == df_elasticidad['rentabilidad_seg3'].max())]
# max_ren_seg4 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg4'] == df_elasticidad['rentabilidad_seg4'].max())]

In [301]:
max_ren_seg0

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,costo,costo_seg0,costo_seg1,costo_seg2,ingresos_seg0,ingresos_seg1,ingresos_seg2,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2
5,18.89,1.706554,1.626123,1.774654,17,4.54617,3.964991,5.079162,4.809673,4.173514,5.39577,0.244725,0.089573,0.113442


In [302]:
max_ren_seg1

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,costo,costo_seg0,costo_seg1,costo_seg2,ingresos_seg0,ingresos_seg1,ingresos_seg2,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2
6,19.47,1.59744,1.528164,1.651946,17,3.770035,3.325011,4.145993,4.017374,3.521907,4.438152,0.232199,0.097146,0.126319


In [303]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg0'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg0['irr']), y=float(max_ing_seg0['ingresos_seg0']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg0['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg0'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg0['irr']), y=float(max_ren_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg0['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 0")


In [304]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg1'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg1['irr']), y=float(max_ing_seg1['ingresos_seg1']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg1['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg1'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg1['irr']), y=float(max_ren_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg1['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 1")

In [305]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg2'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg2['irr']), y=float(max_ing_seg2['ingresos_seg2']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg2['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg2'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg2['irr']), y=float(max_ren_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg2['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 2")

In [276]:
# fig = go.Figure()

# # Se agrega el precio máximo de ingreso
# fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg3'], name='Ingresos ($)'))
# # Se agrega la etiqueta
# fig.add_annotation(x=float(max_ing_seg3['irr']), y=float(max_ing_seg3['ingresos_seg3']),
#                                   text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
#                                   showarrow=True,
#                                   arrowhead=1)
# # Se agrega la recta del precio máximo
# fig.add_vline(x=float(max_ing_seg3['irr']), line_width=2, line_dash="dash",
#                              line_color="blue", opacity=0.25)

# # Se agrega el precio máximo de rentabilidad
# fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg3'], name='Rentabilidad ($)'))
# # Se agrega la etiqueta
# fig.add_annotation(x=float(max_ren_seg3['irr']), y=float(max_ren_seg3['rentabilidad_seg3']),
#                                   text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
#                                   showarrow=True,
#                                   arrowhead=1)
# # Se agrega la recta del precio máximo
# fig.add_vline(x=float(max_ren_seg3['irr']), line_width=2, line_dash="dash",
#                              line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# # Configuración del gráfico
# fig.update_layout(
#     showlegend=True,
#     width=800,
#     height=600,
#     xaxis_title="Precio ($)",
#     yaxis_title="Importe ($)",
#     title="Precios óptimos en Segmento 3")

In [277]:
# fig = go.Figure()

# # Se agrega el precio máximo de ingreso
# fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg4'], name='Ingresos ($)'))
# # Se agrega la etiqueta
# fig.add_annotation(x=float(max_ing_seg3['irr']), y=float(max_ing_seg3['ingresos_seg4']),
#                                   text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
#                                   showarrow=True,
#                                   arrowhead=1)
# # Se agrega la recta del precio máximo
# fig.add_vline(x=float(max_ing_seg3['irr']), line_width=2, line_dash="dash",
#                              line_color="blue", opacity=0.25)

# # Se agrega el precio máximo de rentabilidad
# fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg4'], name='Rentabilidad ($)'))
# # Se agrega la etiqueta
# fig.add_annotation(x=float(max_ren_seg3['irr']), y=float(max_ren_seg3['rentabilidad_seg4']),
#                                   text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
#                                   showarrow=True,
#                                   arrowhead=1)
# # Se agrega la recta del precio máximo
# fig.add_vline(x=float(max_ren_seg3['irr']), line_width=2, line_dash="dash",
#                              line_color="green", opacity=0.25)

# # Agregamos la línea de restricción: Costo
# fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
#                              line_color="orange", opacity=0.5)

# # Configuración del gráfico
# fig.update_layout(
#     showlegend=True,
#     width=800,
#     height=600,
#     xaxis_title="Precio ($)",
#     yaxis_title="Importe ($)",
#     title="Precios óptimos en Segmento 4")

In [278]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg0'], y=df_elasticidad['rentabilidad_seg0'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg0['ingresos_seg0']), y=float(max_ing_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg0['ingresos_seg0']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg0['ingresos_seg0']), y=float(max_ren_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg0['ingresos_seg0']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis=dict(
        title="Ingreso ($)"
       ## ,range=[1, 2]
        ),
   yaxis=dict(
        title="Rentabilidad ($)"
       ##,range=[-0.5, 1]
    ),
    title="Frontera Eficiente Segmento 0")

In [279]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg1'], y=df_elasticidad['rentabilidad_seg1'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg1['ingresos_seg1']), y=float(max_ing_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg1['ingresos_seg1']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg1['ingresos_seg1']), y=float(max_ren_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg1['ingresos_seg1']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
     xaxis=dict(
        title="Ingreso ($)"
      ##  ,range=[1.5, 2.5]
        ),
      yaxis=dict(
        title="Rentabilidad ($)"
       ##,range=[-0.5, 1]
      ),
    title="Frontera Eficiente Segmento 1")

In [280]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg2'], y=df_elasticidad['rentabilidad_seg2'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg2['ingresos_seg2']), y=float(max_ing_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg2['ingresos_seg2']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg2['ingresos_seg2']), y=float(max_ren_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg2['ingresos_seg2']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis_title="Ingreso ($)",
    yaxis_title="Rentabilidad ($)",
    title="Frontera Eficiente Segmento 2")

In [281]:
# fig = go.Figure()

# # Se agrega el precio máximo de ingreso
# fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg3'], y=df_elasticidad['rentabilidad_seg3'], name='Frontera Eficiente'))

# # Se agrega la etiqueta de ingreso
# fig.add_annotation(x=float(max_ing_seg3['ingresos_seg3']), y=float(max_ing_seg3['rentabilidad_seg3']),
#                                   text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
#                                   showarrow=True,
#                                   arrowhead=1)
# # Se agrega la recta del precio máximo
# fig.add_vline(x=float(max_ing_seg3['ingresos_seg3']), line_width=2, line_dash="dash",
#                              line_color="blue", opacity=0.25)

# # Se agrega la etiqueta de rentabilidad
# fig.add_annotation(x=float(max_ren_seg3['ingresos_seg3']), y=float(max_ren_seg3['rentabilidad_seg3']),
#                                   text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
#                                   showarrow=True,
#                                   arrowhead=1)
# # Se agrega la recta del precio máximo
# fig.add_vline(x=float(max_ren_seg3['ingresos_seg3']), line_width=2, line_dash="dash",
#                              line_color="green", opacity=0.25)

# # Configuración del gráfico
# fig.update_layout(
#     showlegend=False,
#     width=800,
#     height=600,
#     xaxis_title="Ingreso ($)",
#     yaxis_title="Rentabilidad ($)",
#     title="Frontera Eficiente Segmento 3")